In [8]:
import pandas as pd 
import numpy as np
import tensorflow as tf


In [9]:
frame=pd.read_csv('Dataset Malware Comma & Space.csv')
frame

,malware,api
0,trojan,getversion createeventa getwindowdc getdevicec...
1,trojan,lstrcpyw getthreadlocale lstrcmpiw globalalloc...
2,trojan,getthreadlocale lstrcmpiw globalalloc globalfr...
3,trojan,localalloc createsemaphorew createsemaphorea g...
4,trojan,getsystemdirectorya isdbcsleadbyte localalloc ...
...,...,...
13351,worm,getversionexw getversionexa getmodulehandlew g...
13352,worm,getsystemdirectorya getversionexw isdbcsleadby...
13353,worm,getversionexw getmodulehandlew getmodulehandle...
13354,worm,getversionexw getmodulehandlew getmodulehandle...


In [10]:
data=frame.sample(frac=1)

In [11]:
train_size=int(len(data)*.8)
print('train_size: ',train_size )
print('test_size: ',len(data)-(train_size))


train_size:  10684
test_size:  2672


In [12]:
data['malware'].value_counts()

trojan      4138
adware      3597
worm        1474
riskware    1219
packed      1104
virus        986
backdoor     838
Name: malware, dtype: int64

In [13]:
def train_test_split(data,train_size):
    train=data[:train_size]
    test=data[train_size:]
    return train , test

In [14]:
train_malware,test_malware=train_test_split(data['malware'], train_size)
train_api , test_api=train_test_split(data['api'],train_size)

In [15]:
from tensorflow import keras
max_words=1000
tokenizer=keras.preprocessing.text.Tokenizer(num_words=max_words,\
                                          char_level=False)

In [16]:
tokenizer.fit_on_texts(train_api)
x_train=tokenizer.texts_to_matrix(train_api)
x_test=tokenizer.texts_to_matrix(test_api)

In [17]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
encoded_y=le.fit_transform(train_malware)
encoded_y_train=keras.utils.to_categorical(encoded_y)
encoded_y=le.fit_transform(test_malware)
encoded_y_test=keras.utils.to_categorical(encoded_y)

In [18]:
x_test.shape

(2672, 1000)

In [19]:
batch_size=10
epochs=5
drop_ratio=0.5

In [20]:
model=tf.keras.models.Sequential([
    tf.keras.layers.Input(shape=(max_words,)),
    tf.keras.layers.Dense(512,activation='relu'),
    tf.keras.layers.Dense(212,activation='relu'),
    tf.keras.layers.Dense(7,activation='softmax')
])

In [21]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [22]:
hist= model.fit(x_train,encoded_y_train,epochs=epochs,batch_size=batch_size,validation_split=0.1)

Train on 9615 samples, validate on 1069 samples
Epoch 1/5
9615/9615 [==============================] - 49s 5ms/sample - loss: 1.0935 - accuracy: 0.6264 - val_loss: 1.0114 - val_accuracy: 0.6586loss: 1.3101 - ac -
Epoch 2/5
9615/9615 [==============================] - 28s 3ms/sample - loss: 0.9542 - accuracy: 0.6729 - val_loss: 0.9711 - val_accuracy: 0.6707
Epoch 3/5
9615/9615 [==============================] - 34s 4ms/sample - loss: 0.9004 - accuracy: 0.6927 - val_loss: 0.9331 - val_accuracy: 0.6838
Epoch 4/5
9615/9615 [==============================] - 44s 5ms/sample - loss: 0.8617 - accuracy: 0.7031 - val_loss: 0.9412 - val_accuracy: 0.6791
Epoch 5/5
9615/9615 [==============================] - 37s 4ms/sample - loss: 0.8308 - accuracy: 0.7120 - val_loss: 0.9306 - val_accuracy: 0.6904


In [23]:
model.evaluate(x_test,encoded_y_test,verbose=2)

2672/1 - 2s - loss: 0.8120 - accuracy: 0.6684


[0.9935315701062094, 0.66841316]

In [24]:
model.evaluate(x_train,encoded_y_train,verbose=2)

10684/1 - 5s - loss: 0.9760 - accuracy: 0.7271


[0.8022018498760267, 0.72706854]

## Hyperprameter Tunning

In [25]:
from tensorflow import keras
Layers = keras.layers
models = keras.models
def run_expriment(batch_size,epochs,drop_ratio):
    print('batch_size: {}, epochs: {}, drop_ratio: {}'.format(batch_size,epochs,drop_ratio))
    model=tf.keras.models.Sequential([
        tf.keras.layers.Input(shape=(max_words,)),
        tf.keras.layers.Dense(512,activation='relu'),
        tf.keras.layers.Dense(212,activation='relu'),
        tf.keras.layers.Dense(7,activation='softmax')
    ])
        
    model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])
    
    hist=model.fit(x_train , encoded_y_train , epochs=epochs , batch_size=batch_size , verbose=0 , validation_split=0.1)
    
    score=model.evaluate(x_test , encoded_y_test , verbose=2 , batch_size=batch_size)
    print('\tTest loss:', score[0])
    print('\tTest accuracy:', score[1])
    

In [26]:
batch_size=16
epochs=10
drop_ratio=0.5
run_expriment(batch_size,epochs,drop_ratio)

batch_size: 16, epochs: 10, drop_ratio: 0.5
2672/1 - 3s - loss: 0.7947 - accuracy: 0.6703
	Test loss: 1.0275757698241823
	Test accuracy: 0.67028445


In [43]:
i=int(input('Please ENter a predected number'))
predection=model.predict(np.array([x_test[i]]))
predection

Please ENter a predected number1


array([[0.0586433 , 0.00790649, 0.00574625, 0.8493915 , 0.02943379,
        0.03319522, 0.0156834 ]], dtype=float32)

In [44]:
x=le.classes_
predection_lable=x[np.argmax(predection)]
predection_lable

'riskware'

In [45]:
print('ACtual_lable:',test_malware.iloc[3])
print('Predected_lable:',predection_lable)

ACtual_lable: trojan
Predected_lable: riskware
